<a href="https://colab.research.google.com/github/PabloJRW/titanic-classifier/blob/main/notebooks/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from joblib import dump

In [49]:
num_features = ['Fare']
cat_features = ['Pclass', 'Sex', 'Embarked']
ff = ['SibSp', 'Parch']

In [38]:
def replace_values(series):
    # Use .loc to select only the values that are not 0 or 1
    mask = ~series.isin([0, 1])
    series.loc[mask] = 2
    return series

In [39]:
# Importing regular expression library
import re

def extract_title(series):
    """
    Extract the social title from name.
    E.g Mr, Mrs, Miss
    """
    pattern = r",\s(.+?)\s"
    df_col = pd.Series(series).fillna("")
    titles = df_col.apply(lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else "")
    return titles

In [50]:
# numerical transformer pipeline (only 'fare')
num_transformer = Pipeline(steps=[
    ('log_transform', FunctionTransformer(np.log1p)),
    ('scaler', StandardScaler( ))]
)

# categorical transformer pipeline
cat_transformer = Pipeline(
    steps=[ 
      ('imputer', SimpleImputer(strategy='most_frequent')),
      ('ohe', OneHotEncoder(handle_unknown='ignore'))]
)

mapper = Pipeline(
    steps=[
        ('mapper', FunctionTransformer(replace_values)),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ])

# 
socialt_transformer = Pipeline(
    steps=[
        ('extractor', FunctionTransformer(extract_title)),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))]
)

# Final pipeline
preprocessing_pipeline = ColumnTransformer(
    transformers=[
      # ('extract', socialt_transformer, ['Name']),
        ('num_transfomer', num_transformer, num_features),
        ('cat_transformer', cat_transformer, cat_features),
        ('mapper_transformer', mapper, ff)
    ],remainder='drop'
)


In [51]:
# importing the training set
url_train = "https://raw.githubusercontent.com/PabloJRW/titanic-classifier/main/datasets/raw/train.csv"
df_train = pd.read_csv(url_train)

In [52]:
df_train_transformed = preprocessing_pipeline.fit_transform(df_train)

KeyError: "None of [Index([('S', 'i', 'b', 'S', 'p'), ('P', 'a', 'r', 'c', 'h')], dtype='object')] are in the [index]"

In [ ]:
df_train_transformed


In [ ]:
# Saving the pipeline using joblib
dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')